In [ ]:
import warnings # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
#import numpy as np
from datetime import datetime, date
import time
from pprint import pprint as pp
#from tqdm import tqdm

from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

# login to myplant()
import dmyplant2 as dmp2
dmp2.cred()
mp = dmp2.MyPlant(0)
dmp2.Engine._list_cached_validations()
#mp._fetch_installed_base(); # refresh local installed fleet database

In [ ]:
def sfun(x):
    #return all([ ("CREYKE BECK" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Yukon Energy" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    return all([ ("Forsa Hartmoor" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("EWB" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Pforzheim" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("BMW Landshut 4.10" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T

In [ ]:
motor = fleet.iloc[2]
modes = ['undefined','OFF','MAN','AUTO']
success = ['success','failed','undefined']
e=dmp2.Engine.from_fleet(mp,motor)
#pp_from='2022-03-28 06:00' # 1 Start
#pp_to='2022-03-28 08:14'
pp_from='2022-08-01'
pp_to='2023-03-01'
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

In [ ]:
# fsm = dmp2.FSMOperator(e, p_from=pp_from, p_to=pp_to)
# fsm.run0(enforce=True, silent=False, debug=False)
# fsm.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
# fsm.run2(silent=False)
# fsm.run4(silent=False)

In [ ]:
#fsm.save_results('Forsa_Hartmoor_M3_debug.dfsm')
fsm = dmp2.FSMOperator.load_results(mp, 'Forsa_Hartmoor_M3_debug.dfsm')

In [ ]:
pp_from = pd.to_datetime(fsm.results['starts'][-3]['starttime'])
pp_to = date.today()

In [ ]:
#pp_from='2023-03-28'
#pp_to='2023-03-29'
fsm2 = dmp2.FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm2.run0(enforce=True, silent=False, debug=False)
fsm2.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
fsm2.run2(silent=False)
fsm2.run4(silent=False)

In [ ]:
print(f"FSM  {fsm.results['starts_counter']} {fsm.results['first_message']} {fsm.results['last_message']}")
print(f"FSM2 {fsm2.results['starts_counter']} {fsm2.results['first_message']} {fsm2.results['last_message']}")

In [ ]:
# pp_from='2022-08-01'
# pp_to='2023-05-04'
# fsm3 = dmp2.FSMOperator(e, p_from=pp_from, p_to=pp_to)
# fsm3.run0(enforce=True, silent=False, debug=False)
# fsm3.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
# fsm3.run2(silent=False)
# fsm3.run4(silent=False)
# print(f"FSM3 {fsm3.results['starts_counter']} {fsm3.results['first_message']} {fsm3.results['last_message']}")

In [ ]:
#fsm3.results
#4.5.2020 validated, Dieter

In [ ]:
start = fsm.results['starts'][0]
for k in start.keys():
    print(f"{k:25} {(str(start[k]) if not hasattr(start[k],'__len__') else '-'):<30} type:{type(start[k])}")

In [ ]:
print(f"SN: {fsm.results['sn']} - {fsm2.results['sn']}")
print(f"starts: {len(fsm.results['starts'])} - {len(fsm2.results['starts'])}")
print(f"stops: {len(fsm.results['stops'])} - {len(fsm2.results['stops'])}")
print(f"messages: {fsm.results['first_message']} - {fsm2.results['first_message']}")
print(f"messages: {fsm.results['last_message']} - {fsm2.results['last_message']}")
print()
for s in fsm.results['starts'][-4:]:
    print(f"Start: no {s['no']} {s['starttime']} {s['mode']}")

print()
for i in range(1,10):
    if fsm.results['starts'][-i]['starttime'] == fsm2.results['starts'][0]['starttime']:
        break

s = fsm.results['starts'][-i]
print(f"{i}: Start: no {s['no']} {s['starttime']} {s['mode']}")

print()
for j, s in enumerate(fsm2.results['starts'][:4]):
    if j < i:
        print('( ', end='')
        print(f"Start: no {s['no']} {s['starttime']} {s['mode']}", end='')
        print(' )')
    else:
        print(f"Start: no {s['no']} {s['starttime']} {s['mode']}")

#---
print('-----')
for s in fsm.results['stops'][-4:]:
    print(f"Stop: no {s['no']} {s['starttime']} {s['mode']}")

print()
for i in range(1,10):
    if fsm.results['stops'][-i]['starttime'] == fsm2.results['stops'][0]['starttime']:
        break

s = fsm.results['stops'][-i]
print(f"{i}: Stop: no {s['no']} {s['starttime']} {s['mode']}")

print()
for j, s in enumerate(fsm2.results['stops'][:4]):
    if j < i:
        print('( ', end='')
        print(f"Stop: no {s['no']} {s['starttime']} {s['mode']}", end='')
        print(' )')
    else:
        print(f"Stop: no {s['no']} {s['starttime']} {s['mode']}")


In [ ]:
fsm.merge_results(fsm2)

In [ ]:
#4.5.2020 validated, Dieter
for s in fsm.results['starts']:
    print(f"Start: no {s['no']:4}  {str(s['starttime']):<35} {s['mode']}")